In [111]:
import glob
from datetime import datetime

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

In [112]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['lines.markersize'] = 10 

path = "top_k_insensitivity"

In [113]:
def clean_measure_name(df_col):
    return df_col\
                .str.replace("IFD_div$", "IFD_div_ori", regex=True)\
                .str.replace("_div", "$_\\div$", regex=False)

def separate_measure_version(df_col):
    return df_col.str.rsplit("_", n=1, expand=True)

In [114]:
def load_result(path):
    list_files = glob.glob(f"{path}/*.pickle")
    
    
    df_all = pd.DataFrame()

    for file in list_files:
      
        dataset = file\
                    .replace(".pickle","")\
                    .split("_")[3]

        df_for_file = pd.DataFrame()
        
        result_dict = pd.read_pickle(file)
        cols = pd.Series(result_dict)

        df_for_file["measure"] = cols.index
        df_for_file["unfairness"] = cols.values
        df_for_file["dataset"] = dataset

        df_for_file["measure"] = clean_measure_name(df_for_file["measure"])
        df_for_file[["measure","version"]] = separate_measure_version(df_for_file["measure"])

        df_for_file[["version","cut-off $k$"]] = df_for_file["version"].str\
                                                    .split("@", expand=True)
        
        df_for_file["cut-off $k$"] = df_for_file["cut-off $k$"].astype(int)

        df_all = pd.concat([df_all, df_for_file], ignore_index=True)

    df_all.dropna(inplace=True)
    df_all.reset_index(drop=True, inplace=True)
 
    return df_all

df_all = load_result(path)

In [115]:
df_all["measure"] = df_all["measure"] + "$_{" + df_all["version"] + "}$"
df_all["measure"] = df_all.measure.str.replace("ori-cut-off}$", "ori}$ + $J_{\ our}}$", regex=False)

list_k = df_all["cut-off $k$"].unique()

In [116]:
selected_index = ["Lastfm", "Amazon-lb", "QK-video", "ML-10M"]

In [117]:
df_all = df_all.set_index(["dataset","measure","version", "cut-off $k$",])
df_all = df_all.reindex(selected_index, level=0)
df_all.reset_index(inplace=True)

In [ ]:
import matplotlib.ticker as ticker

colors = sns.color_palette("colorblind")
list_col = [colors[3], colors[1], colors[2]]

markers = ["X", (4,0,45), "o"]
plotting_kws = dict(x="cut-off $k$", y="unfairness", hue="measure", kind="line", 
                    col="dataset", 
                    style="measure", 
                    palette=list_col,
                    markers=markers, dashes=True, 
                    aspect=1, 
                    height=2.25,
                    )
facet_kws = {
              'sharey': False, 
              "margin_titles":True, 
              "legend_out": True,
              }
grid = sns.relplot(df_all, facet_kws=facet_kws, **plotting_kws)
grid.set_titles(col_template = '{col_name}',size=10)

grid.set(xticks=list_k)
for ax in grid.axes.flatten():
    # ax.xaxis.label.set_size(fontsize=7)
    ax.tick_params(axis='both', which='major', labelsize=7, rotation=0)
    for _, spine in ax.spines.items():
        spine.set_visible(True) 



sns.move_legend(grid, loc="lower center", ncols=3, 
                bbox_to_anchor=(0.525, 1.025), 
                frameon=True,fontsize=10, 
                title=None, markerscale=0.65)

for ha in grid._legend.legendHandles:
    ha.set_mew(0.4)
    ha.set_markeredgecolor("white")

grid.fig.tight_layout(w_pad=0.55)

now = datetime.now()
time = str(now.strftime("%Y-%m-%d_%H%M%S"))
grid.savefig(f"{path}/{path}_{time}.pdf",bbox_inches="tight")   